In [1]:
# Install dependencies
# %pip install numpy
# %pip install -q ipywidgets
# %pip install numba

In [2]:
# Import libraries
import time
import numpy as np
from scipy import signal

from IPython.display import clear_output

from ipywidgets import Layout #, interact, interactive, fixed, interact_manual
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
import ipywidgets as widgets

import matplotlib
import matplotlib.pyplot as plt

# Import local functions
from ising_model import gen_lattice, compute_total_energy, compute_energy_delta, compute_magnetization, compute_magnetization_delta

In [3]:
# Output Caputure
out_sim_res = widgets.Output(layout={"border": "2px solid black"})
out_summary = widgets.Output(layout={"border": "2px solid black"})

In [4]:
# Global constants (Not well understood)
constant_J = 1.0
constant_kB = 1.0

# Some other graphing related configs
cmap = matplotlib.colors.ListedColormap(["Red", "Green"])

In [5]:
# # Function to run the entire simulation for n number of steps
def run_simulation(lattice, J, kB, temp_range, temp_delta, plot_steps, in_cmap):
    current_temp = temp_range[0]
    current_beta = np.Inf if current_temp == 0 else 1.0 / (current_temp * kB)
    total_energy = compute_total_energy(lattice=lattice, int_J=J)
    magnetization = compute_magnetization(lattice=lattice)
    N = lattice.shape[0]

    # Plots and logging variables
    hist_total_engery = [total_energy]
    hist_magnetization = [abs(magnetization / (N**2))]
    n_step = 0

    while current_temp <= temp_range[1]:
        # Select a random state
        flip_coord = np.random.randint(0, N, size = 2)

        # Compute delta
        delta_E = compute_energy_delta(lattice=lattice, flip_coord=flip_coord, int_J=J)
        delta_M = compute_magnetization_delta(lattice=lattice, flip_coord=flip_coord)

        acceptance_prob = min(1, np.exp(-current_beta * delta_E))
        # Perform metropolish checks
        if np.random.random() < acceptance_prob:
            # Accept the flip, and compute new magetization
            lattice[flip_coord[0], flip_coord[1]] *= -1
            total_energy += delta_E
            magnetization += delta_M
        
        # Log results
        hist_total_engery.append(total_energy)
        hist_magnetization.append(abs(magnetization / (N**2)))
        n_step += 1

        # Update temperature if equilibrium reached
        if n_step >= 1_000:
            n_step = 0
            current_temp += temp_delta
            current_beta = np.Inf if current_temp == 0 else 1.0 / (current_temp * kB)
        
        if n_step % plot_steps == 0:
            clear_output(True)
            fig_plot, ax_plot = plt.subplots(1, 3, figsize=(30, 10))
            ax_plot[0].imshow(lattice, interpolation="None", cmap=in_cmap, vmin=-1, vmax=1)
            ax_plot[1].plot(hist_total_engery)
            ax_plot[2].plot(hist_magnetization)
            fig_plot.canvas.draw()
            show_inline_matplotlib_plots()

In [6]:
g_size = widgets.HBox([
    widgets.Label(value="Size of Lattice (N): ", layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")), 
    widgets.IntSlider(
        min=20, 
        max=200, 
        step=1, 
        value=100,
        disabled=False
    )
])
p_ratio = widgets.HBox([
    widgets.Label(value="Ratio of postive spins (+1): ", layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")), 
    widgets.FloatSlider(
        min=0.00, 
        max=1.00, 
        step=0.05, 
        value=0.00,
        disabled=False
    )
])
temp_range = widgets.HBox([
    widgets.Label(value="Temperature Range T: ", layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")), 
    widgets.FloatRangeSlider(
        min=0.00, 
        max=20.00,
        step=0.1,
        value=[0.00, 3.00],
        disabled=False
    )
])
temp_delta = widgets.HBox([
    widgets.Label(value="Temperature Delta: ", layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")), 
    widgets.FloatText(
        value=0.1,
        disabled=False,
        layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")
    )
])
n_steps = widgets.HBox([
    widgets.Label(value="Iterations (x1000): ", layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")), 
    widgets.IntSlider(
        min=10, 
        max=10000,
        step=10,
        value=100,
        disabled=False
    )
])
p_steps = widgets.HBox([
    widgets.Label(value="Steps per plot updated: ", layout=widgets.Layout(display="flex", width="175px", justify_content="flex-end")), 
    widgets.IntSlider(
        min=100, 
        max=10000,
        step=100,
        value=1000,
        disabled=False
    )
])
# Button to allow simulation run
run_btn = widgets.Button(
    description="Run Metropolis!",
    disabled=False,
    button_style="",
    tooltip="Click to begin simulation of metropolis algorithm",
    icon="check",
    layout=widgets.Layout(width="400px", height="auto"),
)
run_btn.style.button_color = "LightGray"

In [7]:
# Button click even handler callback
def button_handler(button_instance):
    # Disable button
    button_instance.disabled = True
    # Disable other widgest
    g_size.children[1].disabled = True
    p_ratio.children[1].disabled = True
    temp_range.children[1].disabled = True
    temp_delta.children[1].disabled = True
    n_steps.children[1].disabled = True

    with out_summary:
        clear_output()

    with out_sim_res:
        clear_output()
        ################################################################
        # Run simulation
        ################################################################
        global lattice, fig_plot, ax_plot
        run_simulation(
            lattice=lattice,
            J=constant_J,
            kB=constant_kB,
            temp_range=temp_range.children[1].value,
            temp_delta=temp_delta.children[1].value,
            plot_steps=p_steps.children[1].value,
            in_cmap=cmap,
        )

    # Re-enable all controls
    n_steps.children[1].disabled = False
    g_size.children[1].disabled = False
    p_ratio.children[1].disabled = False
    temp_range.children[1].disabled = False
    temp_delta.children[1].disabled = False
    button_instance.disabled = False

run_btn.on_click(button_handler)

In [8]:

def setup_lattice(size, p_spin_ratio, dim = 2):
    global lattice, fig_plot, ax_plot
    lattice = gen_lattice(size, p_spin_ratio, dim)
    fig_plot, ax_plot = plt.subplots(1, 1, figsize=(10, 10))
    ax_plot.clear()
    ax_plot.imshow(lattice, interpolation="None", cmap=cmap, vmin=-1, vmax=1)
    fig_plot.canvas.draw()
    show_inline_matplotlib_plots()

In [9]:
out_init = widgets.interactive_output(setup_lattice, {"size": g_size.children[1], "p_spin_ratio": p_ratio.children[1]})
button_box = widgets.HBox(children=[run_btn],layout=widgets.Layout(display='flex', flex_flow='column', align_items='center', width='100%'))
controls = widgets.VBox(children=[g_size, p_ratio, temp_range, temp_delta, n_steps, p_steps, button_box])

out_init.layout = {"border": "2px solid black"}
controls.layout = {"border": "2px solid black"}

column2_stack = widgets.VBox([controls, out_summary])
column2_stack.layout.align_content = "space-around"

final_disp = widgets.VBox([widgets.HBox([out_init, column2_stack]), out_sim_res])
display(final_disp)